# Nettoyage ou modification du dataset

Passage de la version 1 à la version 2 (dynamique) 
Modification au besoin, ajout / suppression / modification des colonnes

In [1]:
import pandas as pd
from datetime import *
import numpy as np

def classCoffee(points):
    if points >= 90:
        # Outstanding
        return 1
    if points <90 and points >= 85:
        # Excellent
        return 2
    if points <85 and points >= 80:
        # Very Good
        return 3 
    else:
        return 4

# Read the CSV file (utf-8 encoded)
df = pd.read_csv('./data/DataRisaralda_v1_utf-8.csv', header=0, sep=',', index_col=0, parse_dates=False, encoding='utf-8')
print(df.shape)


# my bad
df.rename(columns={'EPSG:3116_X': 'EPSG:4326_X', 'EPSG:3116_Y': 'EPSG:4326_Y'}, inplace=True)
df.rename(columns={'CENT_X': 'EPSG:3116_X', 'CENT_Y': 'EPSG:3116_Y'}, inplace=True)

(975, 139)


In [2]:
try:
    df.drop('Cedula', axis=1, inplace=True)
except:
    print("No such column")
    

try:
    df.drop('Fecha Recpt', axis=1, inplace=True)
except:
    print("No such column")
    
    
try:
    df.drop('Factor de Rendimiento', axis=1, inplace=True)
except:
    print("No such column")    
    

In [3]:
cols = ['Malla 17-18','Malla 14-16','Malla <13','Negros y vinagres','Broca','Broca de punto','Veteado','Mordido',
         'Inmaduro','Flojo','Sobresecado','Arrugado','Aplastado','Cristalizado','Reposado','Granizo','Conchas',
         'Partido','Ambar','Defectos totales','Humedad (%)','Merma (%)','Aroma-Fragancia',
         'Acidez','Cuerpo','Sabor',
         'Sabor residual','Dulzor','Uniformidad','Balance','Taza limpia','Puntaje catador','Puntaje Total',
         'prec1','prec2','prec3','prec4','prec5','prec6','prec7','prec8','prec9','prec10','tmin1','tmin2','tmin3','tmin4','tmin5',
         'tmin6','tmin7','tmin8','tmin9','tmin10','tmax1','tmax2','tmax3','tmax4','tmax5','tmax6','tmax7','tmax8','tmax9','tmax10',
         'tmean1','tmean2','tmean3','tmean4','tmean5','tmean6','tmean7','tmean8','tmean9','tmean10','dtr1','dtr2','dtr3','dtr4',
         'dtr5','dtr6','dtr7','dtr8','dtr9','dtr10']
defectos = ['Negros y vinagres','Broca','Broca de punto','Veteado','Mordido',
         'Inmaduro','Flojo','Sobresecado','Arrugado','Aplastado','Cristalizado','Reposado','Granizo','Conchas',
         'Partido','Ambar']

for index, row in df.iterrows():
    date = datetime.strptime(str(row['Fecha Analysis']), '%Y-%m-%d %H:%M:%S')
    df.set_value(index,'Fecha Analysis', date)

# Ajout des moyennes sur les 10 mois de mesures

In [4]:
precs = ['prec1','prec2','prec3','prec4','prec5','prec6','prec7','prec8','prec9','prec10']
tmins = ['tmin1','tmin2','tmin3','tmin4','tmin5','tmin6','tmin7','tmin8','tmin9','tmin10']
tmaxs = ['tmax1','tmax2','tmax3','tmax4','tmax5','tmax6','tmax7','tmax8','tmax9','tmax10']
tmeans = ['tmean1','tmean2','tmean3','tmean4','tmean5','tmean6','tmean7','tmean8','tmean9','tmean10']
dtrs = ['dtr1','dtr2','dtr3','dtr4','dtr5','dtr6','dtr7','dtr8','dtr9','dtr10']

# Moyenne sur les 10 mois de mesures

df['PrecTotalAvg'] = 0.
df['TminTotalAvg'] = 0.
df['TmaxTotalAvg'] = 0.
df['TmeanTotalAvg'] = 0.
df['DtrTotalAvg'] = 0.

df['PrecTotal'] = 0.
df['TminTotal'] = 0.
df['TmaxTotal'] = 0.
df['TmeanTotal'] = 0.
df['DtrTotal'] = 0.


for index, row in df.iterrows():
    
    df.set_value(index,'PrecTotalAvg', row[precs].sum() / len(precs))
    df.set_value(index,'TminTotalAvg', row[tmins].sum() / len(precs))
    df.set_value(index,'TmaxTotalAvg', row[tmaxs].sum() / len(precs))
    df.set_value(index,'TmeanTotalAvg', row[tmeans].sum() / len(precs))
    df.set_value(index,'DtrTotalAvg', row[dtrs].sum() / len(precs))
    
    df.set_value(index,'PrecTotal', row[precs].sum())
    df.set_value(index,'TminTotal', row[tmins].sum())
    df.set_value(index,'TmaxTotal', row[tmaxs].sum())
    df.set_value(index,'TmeanTotal', row[tmeans].sum())
    df.set_value(index,'DtrTotal', row[dtrs].sum())


# Conversion de la luminosité en valeurs numériques


In [5]:
df.LUMINOSID.unique()

array([u'Sol', u'Semi Sombra', u'Sombra'], dtype=object)

In [6]:
def convert(lum):
    if lum == "Sol":
        return 2
    elif lum == "Semi Sombra":
        return 1
    elif lum == "Sombra":
        return 0
    else:
        return 0
    
for index, row in df.iterrows():
    df.set_value(index, 'LUMINOSID', convert(row['LUMINOSID']))
    

# Correction semi-manuelle des erreurs sur les points

In [7]:
catacionpts = ['Aroma-Fragancia','Acidez','Cuerpo','Sabor residual',
               'Sabor','Dulzor','Uniformidad','Balance','Taza limpia','Puntaje catador']

for index, row in df.iterrows():
        if row['Puntaje Total'] > 100:
            print index, row['Puntaje Total']
            df.set_value(index, 'Puntaje catador', 6.9)
            print row[catacionpts].sum()
            
            df.set_value(index, 'Puntaje Total', row[catacionpts].sum())

for index, row in df.iterrows():
        if row['Puntaje Total'] > 100:
            print index, row['Puntaje Total']

for index, row in df.iterrows():
        if np.isnan(row['Acidez']) or np.isnan(row['Puntaje Total']):
            df.set_value(index, catacionpts, 0)
            df.set_value(index, 'Puntaje Total', 0)
        if np.isnan(row['Defectos totales']):
            df.set_value(index, 'Defectos totales', row[defectos].sum())
            


# Convertion SICA

In [8]:
print df['SICA 1'].dtype

df['SICA 1'] = df['SICA 1'].astype(np.int64)
df['SICA 1'] = df['SICA 1'].astype(str)
print df['SICA 1'].dtype



float64
object


In [9]:
df

,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,franco_L3,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,1.0,0.0,0.0,2136.642084,167.842120,259.042801,213.442459,91.200680
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,1.0,0.0,0.0,2108.657956,164.142080,245.656280,204.899179,81.514202
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,1.0,0.0,0.0,2017.611760,176.097761,275.409840,225.753801,99.312080
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,1.0,0.0,1.0,0.0,0.0,1997.817921,182.290040,285.532880,233.911461,103.242836
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,1.0,0.0,0.0,1884.882159,186.833520,292.272481,239.552999,105.438963
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,1.0,0.0,1.0,0.0,0.0,2127.653239,165.337120,251.406281,208.371700,86.069157
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,1.0,0.0,1.0,0.0,0.0,2163.882359,167.005399,255.753720,211.379560,88.748320
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,1.0,0.0,0.0,2034.290001,178.649600,280.121080,229.385340,101.471478
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,1.0,0.0,1.0,0.0,0.0,2072.748882,168.976880,263.126480,216.051681,94.149599
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,1.0,0.0,0.0,2131.279362,166.783280,256.403240,211.593258,89.619961


In [10]:
print type(df.isnull().any())

for a, e in df.isnull().any().iteritems():
    print a, "\t\t", e

<class 'pandas.core.series.Series'>
Municipio 		True
Vereda 		True
Finca 		True
SICA 		True
CEDULA 1 		False
SICA 1 		False
REPRESENTA 		False
VARIEDAD 		False
EPSG:3116_X 		False
EPSG:3116_Y 		False
EPSG:4326_Y 		False
EPSG:4326_X 		False
ASNM 		False
LUMINOSID 		False
Fecha Analysis 		False
Malla 17-18 		True
Malla 14-16 		True
Malla <13 		True
Negros y vinagres 		False
Broca 		False
Broca de punto 		False
Veteado 		False
Mordido 		True
Inmaduro 		False
Flojo 		False
Sobresecado 		False
Arrugado 		False
Aplastado 		False
Cristalizado 		False
Reposado 		False
Granizo 		False
Conchas 		True
Partido 		False
Ambar 		True
Defectos totales 		False
Quaker (Nb Granos) 		True
Fluorescentes (Nb Granos) 		True
Pergamino 		True
Almendra 		True
AlmendraTotal 		True
AlmendraSana 		True
UV 		True
Olor 		True
Humedad (%) 		False
Merma (%) 		False
Aroma-Fragancia 		False
Acidez 		False
Cuerpo 		False
Sabor 		False
Sabor residual 		False
Dulzor 		False
Uniformidad 		False
Balance 		False
Taza limpia 	

# Suppression des doublons

Basé sur la date de test et le numéro SICA

In [11]:
df = df.drop_duplicates(subset=['Fecha Analysis', 'SICA 1'], keep='first', inplace=False)
df = df.reset_index(drop=True)

In [12]:
df
#df[cols] = df[cols].apply(pd.to_numeric)

,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,franco_L3,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,1.0,0.0,0.0,2136.642084,167.842120,259.042801,213.442459,91.200680
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,1.0,0.0,0.0,2108.657956,164.142080,245.656280,204.899179,81.514202
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,1.0,0.0,0.0,2017.611760,176.097761,275.409840,225.753801,99.312080
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,1.0,0.0,1.0,0.0,0.0,1997.817921,182.290040,285.532880,233.911461,103.242836
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,1.0,0.0,0.0,1884.882159,186.833520,292.272481,239.552999,105.438963
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,1.0,0.0,1.0,0.0,0.0,2127.653239,165.337120,251.406281,208.371700,86.069157
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,1.0,0.0,1.0,0.0,0.0,2163.882359,167.005399,255.753720,211.379560,88.748320
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,1.0,0.0,0.0,2034.290001,178.649600,280.121080,229.385340,101.471478
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,1.0,0.0,1.0,0.0,0.0,2072.748882,168.976880,263.126480,216.051681,94.149599
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,1.0,0.0,0.0,2131.279362,166.783280,256.403240,211.593258,89.619961


In [13]:
print(df.shape)


(767, 141)


# Suppression des dégustations avec même SICA à moins de 3 mois d'écart

    Voir: analyse de l'occurence des cafés dans 2_CoffeDataAnalysis


In [14]:

# Certains cafés on été testés plusieurs fois, parfois même 2 fois dans la même année mais à plusieurs mois d'intervale. 

dOccurences = df['SICA 1'].value_counts().to_dict() #.to_frame(name='Occurence')

dfOccurences = pd.DataFrame.from_dict(dOccurences, orient='index')

# Ajout d'une colonne contenant le nombre d'occurence

df['occurence'] = 0

# Fonctionne pas dans datacleaning...?
df['SICA 1'] = df['SICA 1'].astype(str)




In [15]:
for index, row in df.iterrows():
    df.set_value(index, 'occurence', dOccurences.get(str(row['SICA 1'])))

#print df['occurence']

print df['occurence'].value_counts()



df


1    631
2    118
3     18
Name: occurence, dtype: int64


,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal,occurence
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,0.0,0.0,2136.642084,167.842120,259.042801,213.442459,91.200680,1
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,0.0,0.0,2108.657956,164.142080,245.656280,204.899179,81.514202,1
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,0.0,0.0,2017.611760,176.097761,275.409840,225.753801,99.312080,1
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,0.0,1.0,0.0,0.0,1997.817921,182.290040,285.532880,233.911461,103.242836,1
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,0.0,0.0,1884.882159,186.833520,292.272481,239.552999,105.438963,1
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,0.0,1.0,0.0,0.0,2127.653239,165.337120,251.406281,208.371700,86.069157,1
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,0.0,1.0,0.0,0.0,2163.882359,167.005399,255.753720,211.379560,88.748320,1
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,0.0,0.0,2034.290001,178.649600,280.121080,229.385340,101.471478,1
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,0.0,1.0,0.0,0.0,2072.748882,168.976880,263.126480,216.051681,94.149599,1
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,0.0,0.0,2131.279362,166.783280,256.403240,211.593258,89.619961,1


In [16]:
df_1 = df[df['occurence'] == 2]

fields = ['SICA 1','Fecha Analysis','year']




df_1 = df_1[fields].sort_values(['SICA 1'])
df

,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal,occurence
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,0.0,0.0,2136.642084,167.842120,259.042801,213.442459,91.200680,1
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,0.0,0.0,2108.657956,164.142080,245.656280,204.899179,81.514202,1
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,0.0,0.0,2017.611760,176.097761,275.409840,225.753801,99.312080,1
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,0.0,1.0,0.0,0.0,1997.817921,182.290040,285.532880,233.911461,103.242836,1
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,0.0,0.0,1884.882159,186.833520,292.272481,239.552999,105.438963,1
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,0.0,1.0,0.0,0.0,2127.653239,165.337120,251.406281,208.371700,86.069157,1
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,0.0,1.0,0.0,0.0,2163.882359,167.005399,255.753720,211.379560,88.748320,1
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,0.0,0.0,2034.290001,178.649600,280.121080,229.385340,101.471478,1
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,0.0,1.0,0.0,0.0,2072.748882,168.976880,263.126480,216.051681,94.149599,1
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,0.0,0.0,2131.279362,166.783280,256.403240,211.593258,89.619961,1


In [17]:
from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d %H:%M:%S")
    d2 = datetime.strptime(d2, "%Y-%m-%d %H:%M:%S")
    return abs((d2 - d1).days)



In [18]:
item = 1
oldDate = 1
todel = []

for index, row in df_1.iterrows():
    
    if item == 1:
        item = 2
        firstDate = row['Fecha Analysis']
        firstsica = row['SICA 1']
        continue
    if item == 2:
        secondDate = row['Fecha Analysis']
        secondsica = row['SICA 1']
        space = days_between(str(firstDate), str(secondDate))
        if space < 90:
            todel.append(index)
            print("Sica1 {} - Sica2 {}".format(firstsica,secondsica))
            print("Date1 {} - Date2 {}".format(firstDate,secondDate))
        item = 1

print todel

Sica1 6604502461 - Sica2 6604502461
Date1 2016-07-22 00:00:00 - Date2 2016-07-17 00:00:00
Sica1 6608800780 - Sica2 6608800780
Date1 2016-07-22 00:00:00 - Date2 2016-05-24 00:00:00
Sica1 6608800799 - Sica2 6608800799
Date1 2016-07-25 00:00:00 - Date2 2016-09-14 00:00:00
Sica1 6608800899 - Sica2 6608800899
Date1 2016-07-22 00:00:00 - Date2 2016-07-24 00:00:00
Sica1 6608800959 - Sica2 6608800959
Date1 2016-07-24 00:00:00 - Date2 2016-07-21 00:00:00
Sica1 6617000764 - Sica2 6617000764
Date1 2016-05-16 00:00:00 - Date2 2016-07-22 00:00:00
Sica1 6638300445 - Sica2 6638300445
Date1 2016-06-23 00:00:00 - Date2 2016-06-02 00:00:00
Sica1 6644000388 - Sica2 6644000388
Date1 2016-07-22 00:00:00 - Date2 2016-06-23 00:00:00
Sica1 6644000596 - Sica2 6644000596
Date1 2016-07-22 00:00:00 - Date2 2016-05-21 00:00:00
Sica1 6644001113 - Sica2 6644001113
Date1 2016-07-22 00:00:00 - Date2 2016-05-16 00:00:00
Sica1 6645600841 - Sica2 6645600841
Date1 2011-12-07 00:00:00 - Date2 2011-12-29 00:00:00
Sica1 6659

In [19]:
print(len(todel))
print(len(df_1.index))

df = df.drop(todel)
todel = []


21
118


In [20]:
df = df.reset_index(drop=True)

In [21]:
print df.shape

(746, 142)


# Ecriture du dataset V2 

Contient toutes les données (nom de la ferme (Finca) etc)

In [22]:
df.to_csv("./data/DataRisaralda_v2_utf-8.csv", sep=',', encoding='utf-8')
df.to_csv("./data/DataRisaralda_v2_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

In [23]:
df

,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal,occurence
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,0.0,0.0,2136.642084,167.842120,259.042801,213.442459,91.200680,1
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,0.0,0.0,2108.657956,164.142080,245.656280,204.899179,81.514202,1
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,0.0,0.0,2017.611760,176.097761,275.409840,225.753801,99.312080,1
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,0.0,1.0,0.0,0.0,1997.817921,182.290040,285.532880,233.911461,103.242836,1
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,0.0,0.0,1884.882159,186.833520,292.272481,239.552999,105.438963,1
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,0.0,1.0,0.0,0.0,2127.653239,165.337120,251.406281,208.371700,86.069157,1
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,0.0,1.0,0.0,0.0,2163.882359,167.005399,255.753720,211.379560,88.748320,1
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,0.0,0.0,2034.290001,178.649600,280.121080,229.385340,101.471478,1
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,0.0,1.0,0.0,0.0,2072.748882,168.976880,263.126480,216.051681,94.149599,1
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,0.0,0.0,2131.279362,166.783280,256.403240,211.593258,89.619961,1


In [24]:
for col in df.columns:
    print col
    

Municipio
Vereda
Finca
SICA
CEDULA 1
SICA 1
REPRESENTA
VARIEDAD
EPSG:3116_X
EPSG:3116_Y
EPSG:4326_Y
EPSG:4326_X
ASNM
LUMINOSID
Fecha Analysis
Malla 17-18
Malla 14-16
Malla <13
Negros y vinagres
Broca
Broca de punto
Veteado
Mordido
Inmaduro
Flojo
Sobresecado
Arrugado
Aplastado
Cristalizado
Reposado
Granizo
Conchas
Partido
Ambar
Defectos totales
Quaker (Nb Granos)
Fluorescentes (Nb Granos)
Pergamino
Almendra
AlmendraTotal
AlmendraSana
UV
Olor
Humedad (%)
Merma (%)
Aroma-Fragancia
Acidez
Cuerpo
Sabor
Sabor residual
Dulzor
Uniformidad
Balance
Taza limpia
Puntaje catador
Puntaje Total
Taza 1
Taza 2
Taza 3
Taza 4
Taza 5
prec1
prec2
prec3
prec4
prec5
prec6
prec7
prec8
prec9
prec10
tmin1
tmin2
tmin3
tmin4
tmin5
tmin6
tmin7
tmin8
tmin9
tmin10
tmax1
tmax2
tmax3
tmax4
tmax5
tmax6
tmax7
tmax8
tmax9
tmax10
tmean1
tmean2
tmean3
tmean4
tmean5
tmean6
tmean7
tmean8
tmean9
tmean10
dtr1
dtr2
dtr3
dtr4
dtr5
dtr6
dtr7
dtr8
dtr9
dtr10
year
PrecTotalAvg
TminTotalAvg
TmaxTotalAvg
TmeanTotalAvg
DtrTotalAvg
Soi

In [25]:



cols_ = ['SICA 1','EPSG:3116_X','EPSG:3116_Y','EPSG:4326_X','EPSG:4326_Y','Puntaje Total']
df_coordinates_pts = df[cols_]

df_coordinates_pts.to_csv("./data/Coordinates_Points_Risaralda_utf-8.csv", sep=',', encoding='utf-8')
df_coordinates_pts.to_csv("./data/Coordinates_Points_Risaralda_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

# Création d'un set compact contenant la variété et slope / orientation



In [26]:
cols = ['SICA 1','VARIEDAD','ASNM','PrecTotalAvg','TminTotalAvg','TmaxTotalAvg','TmeanTotalAvg','DtrTotalAvg',
        "LUMINOSID", "pH_avg","org_avg","year","Defectos totales","Puntaje Total",
       "franco_L1","arcilloso_L1","limoso_L1","arenoso_L1","cascajoso_L1",
        "franco_L2","arcilloso_L2","limoso_L2","arenoso_L2","cascajoso_L2",
        "franco_L3","arcilloso_L3","limoso_L3","arenoso_L3","cascajoso_L3"]

francos = ["franco_L1","franco_L2","franco_L3"]
arcillosos = ["arcilloso_L1","arcilloso_L2","arcilloso_L3"]
limosos = ["limoso_L1","limoso_L2","limoso_L3"]
arenosos = ["arenoso_L1","arenoso_L2","arenoso_L3"]
cascajosos = ["cascajoso_L1","cascajoso_L2","cascajoso_L3"]

df_compact_1 = df[cols].copy(deep=True)

print df_compact_1

# Ajout de colonnes pour résumer la texture du sol.
df_compact_1["Franco"] = 0
df_compact_1["Arcilloso"] = 0
df_compact_1["Limoso"] = 0
df_compact_1["Arenoso"] = 0
df_compact_1["Cascajoso"] = 0

# df.set_value(index,'PrecTotalAvg', row[precs].sum() / len(precs))

for index, row in df_compact_1.iterrows():
    df_compact_1.set_value(index, 'Franco', row[francos].sum())
    df_compact_1.set_value(index, 'Arcilloso', row[arcillosos].sum())
    df_compact_1.set_value(index, 'Limoso', row[limosos].sum())
    df_compact_1.set_value(index, 'Arenoso', row[arenosos].sum())
    df_compact_1.set_value(index, 'Cascajoso', row[cascajosos].sum())

sols = ["franco_L1","arcilloso_L1","limoso_L1","arenoso_L1","cascajoso_L1",
        "franco_L2","arcilloso_L2","limoso_L2","arenoso_L2","cascajoso_L2",
        "franco_L3","arcilloso_L3","limoso_L3","arenoso_L3","cascajoso_L3"]


for col in sols:
    df_compact_1.drop(col, axis=1, inplace=True)

df_compact_1['Category'] = 0

for index, row in df_compact_1.iterrows():
    df_compact_1.set_value(index, 'Category', classCoffee(row['Puntaje Total']))

df_compact_1.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df_compact_1.rename(columns={'SICA 1': 'SICA'}, inplace=True)
df_compact_1.rename(columns={'LUMINOSID': 'Luminosidad'}, inplace=True)
df_compact_1.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df_compact_1.rename(columns={'Puntaje Total': 'PuntajeTotal'}, inplace=True)






         SICA 1  VARIEDAD    ASNM  PrecTotalAvg  TminTotalAvg  TmaxTotalAvg  \
0    6607500996  COLOMBIA  1536.0    213.664208     16.784212     25.904280   
1    6607500981  COLOMBIA  1650.0    210.865796     16.414208     24.565628   
2    6607501034  COLOMBIA  1444.0    201.761176     17.609776     27.540984   
3    6607500081    T?PICA  1344.0    199.781792     18.229004     28.553288   
4    6607500108  COLOMBIA  1354.0    188.488216     18.683352     29.227248   
5    6607500309  COLOMBIA  1568.0    212.765324     16.533712     25.140628   
6    6607500014  COLOMBIA  1505.0    216.388236     16.700540     25.575372   
7    6607500148  COLOMBIA  1318.0    203.429000     17.864960     28.012108   
8    6607500516  COLOMBIA  1552.0    207.274888     16.897688     26.312648   
9    6607501187  COLOMBIA  1454.0    213.127936     16.678328     25.640324   
10   6607500304  CASTILLO  1593.0    213.127936     16.678328     25.640324   
11   6607500533  COLOMBIA  1360.0    210.137280     

"df_compact_1['PrecTotalAvg'] = df_compact_1['PrecTotalAvg'].astype(float)\ndf_compact_1['TminTotalAvg'] = df_compact_1['TminTotalAvg'].astype(float)\ndf_compact_1['TmaxTotalAvg'] = df_compact_1['TmaxTotalAvg'].astype(float)\ndf_compact_1['TmeanTotalAvg'] = df_compact_1['TmeanTotalAvg'].astype(float)\ndf_compact_1['DtrTotalAvg'] = df_compact_1['DtrTotalAvg'].astype(float)"

In [27]:
## Adding slope and orientation 

 
df_temp = pd.read_csv('./data/Slope_Orientation/Risaralda_orientation_slope.csv', header=0, sep=',', index_col=0, parse_dates=False, encoding='utf-8')
#df_temp = df_temp.drop('FID', 1)


cols = ["SICA_1","ris_aspect","ris_slope"]
df_temp = df_temp[cols]

df_temp.rename(columns={'SICA_1': 'SICA'}, inplace=True)
df_temp.rename(columns={'ris_aspect': 'OrientationNum'}, inplace=True)
df_temp.rename(columns={'ris_slope': 'Slope'}, inplace=True)

df_temp['SICA'] = df_temp['SICA'].round(0).astype(np.int64)
df_temp['SICA'] = df_temp['SICA'].astype(str)

#print df_temp

print df_temp.shape
print df_compact_1.shape


d_orient = dict(zip(df_temp.SICA, df_temp.OrientationNum))
d_slope = dict(zip(df_temp.SICA, df_temp.Slope))


print d_slope.get('6607500996')

df_compact_1['OrientationNum'] = 0.
df_compact_1['Slope'] = 0.

for index, row in df_compact_1.iterrows():
    df_compact_1.set_value(index, 'OrientationNum', d_orient.get(row['SICA']))
    df_compact_1.set_value(index, 'Slope', d_slope.get(row['SICA']))


(746, 3)
(746, 20)
32.1693993


In [28]:
# Transofrm orientation in categorical


def convert_orientation(deg, normal = True):
    if(not normal):
        if deg == 0:
            return np.nan
        else:
            return deg
        
    if ((deg>0 and deg<22.5) or (deg > 337.5)):
        return "North"
    if (deg > 22.5 and deg <= 67.5):
        return "Northeast"
    if (deg > 67.5 and deg <= 112.5):
        return "East"
    if (deg > 112.5 and deg <= 157.5):
        return "Southeast"
    if (deg > 157.5 and deg <= 202.5):
        return "South"
    if (deg > 202.5 and deg <= 247.5):
        return "Southwest"
    if (deg > 247.5 and deg <= 292.5):
        return "West"
    if (deg > 292.5 and deg <= 337.5):
        return "Northwest"
    else:
        return np.nan

def convert_slope(slope):
    if slope == 0:
        return np.nan
    else:
        return slope

df_compact_1['Orientation'] = ""

df_compact_2 = df_compact_1.copy(deep=True)

for index, row in df_compact_1.iterrows():
    df_compact_1.set_value(index, 'Orientation', convert_orientation(row['OrientationNum']))
    df_compact_1.set_value(index, 'Slope', convert_slope(row['Slope']))

for index, row in df_compact_2.iterrows():
    df_compact_2.set_value(index, 'Orientation', convert_orientation(row['OrientationNum'], normal = False))
    df_compact_2.set_value(index, 'Slope', convert_slope(row['Slope']))
    

df_compact_2.drop('OrientationNum', axis=1, inplace=True)

In [29]:
df_compact_1

,SICA,VARIEDAD,ASNM,PrecTotalAvg,TminTotalAvg,TmaxTotalAvg,TmeanTotalAvg,DtrTotalAvg,Luminosidad,pH_avg,...,PuntajeTotal,Franco,Arcilloso,Limoso,Arenoso,Cascajoso,Category,OrientationNum,Slope,Orientation
0,6607500996,COLOMBIA,1536.0,213.664208,16.784212,25.904280,21.344246,9.120068,2,5.625000,...,82.000,3,3,3,0,0,3,114.749000,32.169399,Southeast
1,6607500981,COLOMBIA,1650.0,210.865796,16.414208,24.565628,20.489918,8.151420,2,5.625000,...,79.000,3,3,3,0,0,4,139.485001,16.335199,Southeast
2,6607501034,COLOMBIA,1444.0,201.761176,17.609776,27.540984,22.575380,9.931208,1,5.350000,...,78.500,3,3,3,0,1,4,290.726013,32.662201,West
3,6607500081,T?PICA,1344.0,199.781792,18.229004,28.553288,23.391146,10.324284,2,5.400000,...,80.000,3,0,3,0,0,3,116.565002,32.379101,Southeast
4,6607500108,COLOMBIA,1354.0,188.488216,18.683352,29.227248,23.955300,10.543896,2,5.350000,...,75.500,3,3,3,0,1,4,178.363007,29.570299,South
5,6607500309,COLOMBIA,1568.0,212.765324,16.533712,25.140628,20.837170,8.606916,2,5.400000,...,81.000,3,0,3,0,0,3,94.497200,19.883400,East
6,6607500014,COLOMBIA,1505.0,216.388236,16.700540,25.575372,21.137956,8.874832,2,5.400000,...,76.375,3,0,3,0,0,4,11.768300,30.790400,North
7,6607500148,COLOMBIA,1318.0,203.429000,17.864960,28.012108,22.938534,10.147148,2,5.625000,...,74.750,3,3,3,0,0,4,37.504101,23.709801,Northeast
8,6607500516,COLOMBIA,1552.0,207.274888,16.897688,26.312648,21.605168,9.414960,2,5.400000,...,79.000,3,0,3,0,0,4,93.945198,13.253100,East
9,6607501187,COLOMBIA,1454.0,213.127936,16.678328,25.640324,21.159326,8.961996,2,5.625000,...,79.000,3,3,3,0,0,4,183.600998,30.133499,South


In [30]:
df_compact_2

,SICA,VARIEDAD,ASNM,PrecTotalAvg,TminTotalAvg,TmaxTotalAvg,TmeanTotalAvg,DtrTotalAvg,Luminosidad,pH_avg,...,DefectosTotales,PuntajeTotal,Franco,Arcilloso,Limoso,Arenoso,Cascajoso,Category,Slope,Orientation
0,6607500996,COLOMBIA,1536.0,213.664208,16.784212,25.904280,21.344246,9.120068,2,5.625000,...,3.270000,82.000,3,3,3,0,0,3,32.169399,114.749
1,6607500981,COLOMBIA,1650.0,210.865796,16.414208,24.565628,20.489918,8.151420,2,5.625000,...,3.370000,79.000,3,3,3,0,0,4,16.335199,139.485
2,6607501034,COLOMBIA,1444.0,201.761176,17.609776,27.540984,22.575380,9.931208,1,5.350000,...,2.490000,78.500,3,3,3,0,1,4,32.662201,290.726
3,6607500081,T?PICA,1344.0,199.781792,18.229004,28.553288,23.391146,10.324284,2,5.400000,...,2.720000,80.000,3,0,3,0,0,3,32.379101,116.565
4,6607500108,COLOMBIA,1354.0,188.488216,18.683352,29.227248,23.955300,10.543896,2,5.350000,...,5.960000,75.500,3,3,3,0,1,4,29.570299,178.363
5,6607500309,COLOMBIA,1568.0,212.765324,16.533712,25.140628,20.837170,8.606916,2,5.400000,...,2.600000,81.000,3,0,3,0,0,3,19.883400,94.4972
6,6607500014,COLOMBIA,1505.0,216.388236,16.700540,25.575372,21.137956,8.874832,2,5.400000,...,2.780000,76.375,3,0,3,0,0,4,30.790400,11.7683
7,6607500148,COLOMBIA,1318.0,203.429000,17.864960,28.012108,22.938534,10.147148,2,5.625000,...,2.670000,74.750,3,3,3,0,0,4,23.709801,37.5041
8,6607500516,COLOMBIA,1552.0,207.274888,16.897688,26.312648,21.605168,9.414960,2,5.400000,...,1.300000,79.000,3,0,3,0,0,4,13.253100,93.9452
9,6607501187,COLOMBIA,1454.0,213.127936,16.678328,25.640324,21.159326,8.961996,2,5.625000,...,1.550000,79.000,3,3,3,0,0,4,30.133499,183.601


In [31]:
# Reorder and write



cols_reorder = ["SICA","year","VARIEDAD","ASNM",'Slope','Orientation',
                            "Franco","Arcilloso","Limoso","Arenoso","Cascajoso","pH_avg","org_avg",
                            'PrecTotalAvg','TminTotalAvg','TmaxTotalAvg','TmeanTotalAvg','DtrTotalAvg',
                            "Luminosidad","DefectosTotales","PuntajeTotal", "Category"]
df_compact_1 = df_compact_1[cols_reorder]
df_compact_2 = df_compact_2[cols_reorder]

df_compact_1.to_csv("./data/df_compact_1_utf-8.csv", sep=',', encoding='utf-8')
df_compact_1.to_csv("./data/df_compact_1_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

df_compact_2.to_csv("./data/df_compact_2_utf-8.csv", sep=',', encoding='utf-8')
df_compact_2.to_csv("./data/df_compact_2_iso-8859-1.csv", sep=',', encoding='iso-8859-1')


print df_compact_1.columns

Index([           u'SICA',            u'year',        u'VARIEDAD',
                  u'ASNM',           u'Slope',     u'Orientation',
                u'Franco',       u'Arcilloso',          u'Limoso',
               u'Arenoso',       u'Cascajoso',          u'pH_avg',
               u'org_avg',    u'PrecTotalAvg',    u'TminTotalAvg',
          u'TmaxTotalAvg',   u'TmeanTotalAvg',     u'DtrTotalAvg',
           u'Luminosidad', u'DefectosTotales',    u'PuntajeTotal',
              u'Category'],
      dtype='object')


In [32]:
df_compact_1

,SICA,year,VARIEDAD,ASNM,Slope,Orientation,Franco,Arcilloso,Limoso,Arenoso,...,org_avg,PrecTotalAvg,TminTotalAvg,TmaxTotalAvg,TmeanTotalAvg,DtrTotalAvg,Luminosidad,DefectosTotales,PuntajeTotal,Category
0,6607500996,2011,COLOMBIA,1536.0,32.169399,Southeast,3,3,3,0,...,0.021750,213.664208,16.784212,25.904280,21.344246,9.120068,2,3.270000,82.000,3
1,6607500981,2011,COLOMBIA,1650.0,16.335199,Southeast,3,3,3,0,...,0.021750,210.865796,16.414208,24.565628,20.489918,8.151420,2,3.370000,79.000,4
2,6607501034,2011,COLOMBIA,1444.0,32.662201,West,3,3,3,0,...,0.052333,201.761176,17.609776,27.540984,22.575380,9.931208,1,2.490000,78.500,4
3,6607500081,2011,T?PICA,1344.0,32.379101,Southeast,3,0,3,0,...,0.038000,199.781792,18.229004,28.553288,23.391146,10.324284,2,2.720000,80.000,3
4,6607500108,2011,COLOMBIA,1354.0,29.570299,South,3,3,3,0,...,0.052333,188.488216,18.683352,29.227248,23.955300,10.543896,2,5.960000,75.500,4
5,6607500309,2011,COLOMBIA,1568.0,19.883400,East,3,0,3,0,...,0.038000,212.765324,16.533712,25.140628,20.837170,8.606916,2,2.600000,81.000,3
6,6607500014,2011,COLOMBIA,1505.0,30.790400,North,3,0,3,0,...,0.038000,216.388236,16.700540,25.575372,21.137956,8.874832,2,2.780000,76.375,4
7,6607500148,2011,COLOMBIA,1318.0,23.709801,Northeast,3,3,3,0,...,0.021750,203.429000,17.864960,28.012108,22.938534,10.147148,2,2.670000,74.750,4
8,6607500516,2011,COLOMBIA,1552.0,13.253100,East,3,0,3,0,...,0.038000,207.274888,16.897688,26.312648,21.605168,9.414960,2,1.300000,79.000,4
9,6607501187,2011,COLOMBIA,1454.0,30.133499,South,3,3,3,0,...,0.021750,213.127936,16.678328,25.640324,21.159326,8.961996,2,1.550000,79.000,4


# Création d 'un set purement numérique

Attention certain champs comme "quarker" ou "fluorescentes" ont été supprimé étant donné qu'ils ne sont pas complet pour toutes les entrées.

Ci-dessous les colonnes que l'on souhaite voir dans DataRisaralda_v2Numeric_utf-8.csv 

In [33]:
# Inputs: SICA, climat, soil
# Output: Physical caracteristics, degustation

cols = ['SICA 1',
        'prec1','prec2','prec3','prec4','prec5','prec6','prec7','prec8','prec9','prec10',
        'tmin1','tmin2','tmin3','tmin4','tmin5','tmin6','tmin7','tmin8','tmin9','tmin10',
        'tmax1','tmax2','tmax3','tmax4','tmax5','tmax6','tmax7','tmax8','tmax9','tmax10',
        'tmean1','tmean2','tmean3','tmean4','tmean5','tmean6','tmean7','tmean8','tmean9','tmean10',
        'dtr1','dtr2','dtr3','dtr4','dtr5','dtr6','dtr7','dtr8','dtr9','dtr10',
        'PrecTotalAvg','TminTotalAvg','TmaxTotalAvg','TmeanTotalAvg','DtrTotalAvg',
        'PrecTotal','TminTotal','TmaxTotal','TmeanTotal','DtrTotal',
        'year','ASNM', 'LUMINOSID',
        "pH_avg","org_avg",
        "franco_L1","arcilloso_L1","limoso_L1","arenoso_L1","cascajoso_L1",
        "franco_L2","arcilloso_L2","limoso_L2","arenoso_L2","cascajoso_L2",
        "franco_L3","arcilloso_L3","limoso_L3","arenoso_L3","cascajoso_L3",
        'Malla 17-18','Malla 14-16','Malla <13','Negros y vinagres','Broca','Broca de punto','Veteado','Mordido',
        'Inmaduro','Flojo','Sobresecado','Arrugado','Aplastado','Cristalizado','Reposado','Granizo','Conchas',
        'Partido','Ambar','Defectos totales','Humedad (%)','Merma (%)','Aroma-Fragancia',
        'Acidez','Cuerpo','Sabor','Sabor residual','Dulzor','Uniformidad','Balance','Taza limpia','Puntaje catador','Puntaje Total']

In [34]:
df_num = df[cols]

In [35]:
type(df_num['pH_avg'][2])

numpy.float64

In [36]:
df_num.to_csv("./data/DataRisaralda_v2Numeric_utf-8.csv", sep=',', encoding='utf-8')
df_num.to_csv("./data/DataRisaralda_v2Numeric_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

# Dataset numérique complet
    sans données manquantes au niveau des colonnes !
    Les colonnes supprimées sont indiquées ci-dessous

In [37]:
print df_num.shape
colsdfnum = df_num.columns

(746, 114)


In [38]:
df_num_complete = df_num.dropna(axis=1, how='any')
print df_num_complete.shape
colsdfnumcomplete = df_num_complete.columns

(746, 108)


In [39]:
deleted_cols = list(set(colsdfnum) - set(colsdfnumcomplete))
print("Colonnes supprimées: {}".format(deleted_cols))

Colonnes supprimées: [u'Malla 14-16', u'Conchas', u'Malla 17-18', u'Malla <13', u'Mordido', u'Ambar']


# Renomage des colonnes

Attention à vérifier dans les scripts précédents

In [40]:
df_num_complete.rename(columns={'Negros y vinagres': 'NegrosYVinagres'}, inplace=True)
df_num_complete.rename(columns={'Broca de punto': 'BrocaDePunto'}, inplace=True)
df_num_complete.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df_num_complete.rename(columns={'Humedad (%)': 'Humedad'}, inplace=True)
df_num_complete.rename(columns={'Merma (%)': 'Merma'}, inplace=True)
df_num_complete.rename(columns={'Sabor residual': 'SaborResidual'}, inplace=True)
df_num_complete.rename(columns={'Taza limpia': 'TazaLimpia'}, inplace=True)
df_num_complete.rename(columns={'Puntaje catador': 'PuntajeCatador'}, inplace=True)
df_num_complete.rename(columns={'Puntaje Total': 'PuntajeTotal'}, inplace=True)
df_num_complete.rename(columns={'SICA 1': 'SICA'}, inplace=True)


C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [41]:
df_num_complete.to_csv("./data/DataRisaralda_v2Numeric_Complete_utf-8.csv", sep=',', encoding='utf-8')
df_num_complete.to_csv("./data/DataRisaralda_v2Numeric_Complete_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

In [42]:
print(df_num_complete.shape)

(746, 108)


# Insertion de la classe du café (sous forme numérique)

|Total Score|       Quality Classification||
|---|---|---|
|90-100	|          Outstanding|	             Specialty|
|85-89.99	|      Excellent||
|80-84.99|	      Very Good||
|<80.0	|          Below Specialty Quality|	 Not Specialty|


http://www.scaa.org/?page=resources&d=cupping-protocols

In [43]:
df_num_complete_class = df_num_complete.copy(deep=True)
df_num_complete_class['category'] = 0

for index, row in df_num_complete_class.iterrows():
    df_num_complete_class.set_value(index, 'category', classCoffee(row['PuntajeTotal']))

In [44]:
print df_num_complete_class.columns
for e in df_num_complete_class['category']:
    print e

Index([          u'SICA',          u'prec1',          u'prec2',
                u'prec3',          u'prec4',          u'prec5',
                u'prec6',          u'prec7',          u'prec8',
                u'prec9',
       ...
               u'Cuerpo',          u'Sabor',  u'SaborResidual',
               u'Dulzor',    u'Uniformidad',        u'Balance',
           u'TazaLimpia', u'PuntajeCatador',   u'PuntajeTotal',
             u'category'],
      dtype='object', length=109)
3
4
4
3
4
3
4
4
4
4
4
4
3
3
4
4
3
4
4
3
4
4
3
4
4
4
3
3
4
4
4
4
4
4
4
3
4
4
3
3
4
4
3
4
4
3
4
4
3
4
4
3
4
4
4
3
4
3
3
4
4
3
3
3
4
3
4
3
4
4
4
4
4
3
4
4
3
3
3
4
4
4
4
4
4
4
3
4
3
3
4
2
3
4
4
4
4
3
2
4
4
3
4
3
3
4
4
3
3
4
4
4
3
3
4
3
3
3
3
4
3
4
4
4
3
4
3
4
4
4
3
3
4
4
3
3
3
4
4
4
3
3
3
4
4
4
4
4
4
4
3
4
4
3
3
3
4
4
4
4
3
4
4
4
4
3
3
3
4
4
4
3
4
3
3
3
4
3
4
4
3
3
4
3
3
3
3
3
3
3
4
4
4
4
4
3
3
4
3
4
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
3
4
4
4
4
4
3
4
3
4
4
3
4
4
3
4
4
4
3
4
3
3
4
3
4
4
3
3
3
3
4
4
3
3
3
4
4
4
3
3
4


## Utilisé spécialement dans R 
Remplacement des noms avec espaces -> trim

In [45]:
df_num_complete_class.to_csv("./data/DataRisaralda_v2Numeric_Complete_class_utf-8.csv", sep=',', encoding='utf-8')
df_num_complete_class.to_csv("./data/DataRisaralda_v2Numeric_Complete_class_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

In [46]:
df_num_complete_class

,SICA,prec1,prec2,prec3,prec4,prec5,prec6,prec7,prec8,prec9,...,Cuerpo,Sabor,SaborResidual,Dulzor,Uniformidad,Balance,TazaLimpia,PuntajeCatador,PuntajeTotal,category
0,6607500996,145.655601,205.839480,257.519199,195.011120,200.143320,222.754401,184.180201,126.713681,272.889601,...,7.25,7.50,7.50,10.0,10.00,7.25,10.0,7.50,82.000,3
1,6607500981,147.673640,193.916400,257.884318,197.673759,200.862363,217.880001,177.609679,128.226520,265.137279,...,7.00,7.00,7.00,10.0,10.00,7.00,10.0,7.00,79.000,4
2,6607501034,138.982079,205.851759,242.809320,178.794561,189.439719,201.342642,173.090201,121.329240,260.522919,...,7.00,7.00,7.00,10.0,10.00,7.00,10.0,6.50,78.500,4
3,6607500081,139.238720,205.736320,235.844480,178.932759,193.465879,199.170641,174.635161,127.586880,251.448082,...,7.00,7.00,7.00,10.0,10.00,7.00,10.0,7.50,80.000,3
4,6607500108,135.702361,196.869080,225.344479,170.141559,185.258480,178.752480,160.563280,121.434240,236.598439,...,6.50,7.00,6.50,10.0,10.00,6.25,10.0,6.25,75.500,4
5,6607500309,146.721480,200.230720,258.594282,196.977960,200.468881,221.469279,180.504159,126.448520,270.146396,...,7.25,7.50,7.25,10.0,10.00,7.00,10.0,7.25,81.000,3
6,6607500014,146.140280,203.069520,257.035401,199.094400,202.996001,235.458680,189.824600,132.809239,270.253398,...,6.50,6.75,6.75,10.0,10.00,6.50,10.0,6.50,76.375,4
7,6607500148,141.535240,208.818561,242.029041,182.528279,195.608999,201.077200,177.365961,125.745360,259.044361,...,6.25,6.50,6.50,10.0,10.00,6.25,10.0,6.00,74.750,4
8,6607500516,143.263719,205.893320,254.750961,187.448801,193.410159,203.332360,174.027519,118.543120,271.275042,...,7.00,7.00,7.00,10.0,10.00,7.00,10.0,7.00,79.000,4
9,6607501187,146.046481,204.142081,258.391841,195.522799,199.892919,220.659438,182.003241,125.532681,272.582480,...,7.00,7.00,7.00,10.0,10.00,7.00,10.0,7.00,79.000,4


# Création d'un set GPS - Class

In [47]:


df_coords_class = df_coordinates_pts.copy(deep=True)

df_coords_class['category'] = 0

for index, row in df_coords_class.iterrows():
    df_coords_class.set_value(index, 'category', classCoffee(row['Puntaje Total']))
del df_coords_class['Puntaje Total']

In [48]:
df_coords_class.to_csv("./data/Coordinates_Class_Risaralda_utf-8.csv", sep=',', encoding='utf-8')
df_coords_class.to_csv("./data/Coordinates_Class_Risaralda_iso-8859-1.csv", sep=',', encoding='iso-8859-1')
df_coords_class

,SICA 1,EPSG:3116_X,EPSG:3116_Y,EPSG:4326_X,EPSG:4326_Y,category
0,6607500996,790897.6106,1037547.461,-75.962506,4.933069,3
1,6607500981,790598.7547,1037800.494,-75.965206,4.935348,4
2,6607501034,789390.5633,1038560.999,-75.976113,4.942191,4
3,6607500081,792672.5954,1042507.264,-75.946637,4.977942,3
4,6607500108,792341.9870,1041185.481,-75.949583,4.965987,4
5,6607500309,789867.6532,1036981.771,-75.971773,4.927930,3
6,6607500014,791308.3376,1041652.777,-75.958910,4.970184,4
7,6607500148,794450.1679,1039293.955,-75.930536,4.948944,4
8,6607500516,789741.6682,1033348.814,-75.972816,4.895092,4
9,6607501187,790532.0700,1036504.194,-75.965774,4.923631,4
